<a href="https://colab.research.google.com/github/MS-CONQUEROR/RAG-Projects/blob/main/RAG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain==0.3.27 langchain-community==0.3.31 langchain-groq==0.3.8 langchain-google-genai==2.1.12 chromadb==1.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
import json

with open("cricket_rag_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(data.keys())
print(data["documents"][0])
print(data)

dict_keys(['documents', 'metadata'])
{'id': 'doc_001', 'title': 'History of Cricket', 'content': 'Cricket originated in England in the 16th century and became an international sport by the 19th century. The International Cricket Council (ICC) was founded in 1909 to govern the game worldwide.'}
{'documents': [{'id': 'doc_001', 'title': 'History of Cricket', 'content': 'Cricket originated in England in the 16th century and became an international sport by the 19th century. The International Cricket Council (ICC) was founded in 1909 to govern the game worldwide.'}, {'id': 'doc_002', 'title': 'Cricket Formats', 'content': 'There are three major formats of cricket: Test cricket, One Day Internationals (ODIs), and Twenty20 (T20). Test cricket is played over five days, while ODIs and T20s are limited-overs formats.'}, {'id': 'doc_003', 'title': 'Legendary Cricketers', 'content': 'Sachin Tendulkar is the highest run-scorer in international cricket. Don Bradman holds the highest Test batting av

In [ ]:
from langchain.schema import Document
docs = [
    Document(
        page_content=d["content"],       # main text
        metadata={"id": d["id"], "title": d["title"]}  # optional
    )
    for d in data["documents"]
]
for d in docs:
    print(d)

page_content='Cricket originated in England in the 16th century and became an international sport by the 19th century. The International Cricket Council (ICC) was founded in 1909 to govern the game worldwide.' metadata={'id': 'doc_001', 'title': 'History of Cricket'}
page_content='There are three major formats of cricket: Test cricket, One Day Internationals (ODIs), and Twenty20 (T20). Test cricket is played over five days, while ODIs and T20s are limited-overs formats.' metadata={'id': 'doc_002', 'title': 'Cricket Formats'}
page_content='Sachin Tendulkar is the highest run-scorer in international cricket. Don Bradman holds the highest Test batting average of 99.94. Muttiah Muralitharan is the highest wicket-taker in Test cricket.' metadata={'id': 'doc_003', 'title': 'Legendary Cricketers'}
page_content='The ICC Cricket World Cup was first held in 1975. Australia has won the most ODI World Cups. The ICC T20 World Cup began in 2007.' metadata={'id': 'doc_004', 'title': 'Cricket World Cu

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GEMINI_API_KEY)

In [ ]:
from langchain.vectorstores import Chroma
vector_store = Chroma(
    embedding_function = embeddings,
    persist_directory = "chroma_db",
    collection_name = "cricket_rag"
)
vector_store.add_documents(docs)

/tmp/ipython-input-964999369.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


['2496c89e-ffe0-40b0-8e9a-4bc8635f81aa',
 'f86e9541-74d4-4a3b-a087-67df44c63927',
 '03c8caf3-278b-4cb4-aae6-7f6d4697e6bf',
 'd476b6de-40cc-4d61-a38b-c38f37af616a',
 '8b488682-b683-4e60-bec5-6a17327c2a68']

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata
groq2 = userdata.get('GROQ_API_KEY')
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    api_key=groq2,
    temperature=0.3,
    max_tokens=200
)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
while True:
  print("Enter exit to stop")
  query = input("Enter a query: ")
  if query == "exit":
    break
  response = qa_chain.invoke({"query": query})
  response = qa_chain.invoke({"query": query})
  print("Query:", query)
  print("Answer:", response["result"])
print("\nSources:", [doc.metadata for doc in response["source_documents"]])


Enter exit to stop
Enter a query: exit

Sources: [{'title': 'Legendary Cricketers', 'id': 'doc_003'}, {'id': 'doc_005', 'title': 'Important Cricket Records'}]
